In [48]:

from __future__ import division, print_function
import sympy as sm
from sympy import trigsimp
from sympy import latex

import matplotlib.pyplot as plt
import sympy.physics.mechanics as me
import mpmath as mp
import symbtools as st
import numpy as np
import symbtools.modeltools as mt
import symbtools.noncommutativetools as nct
from scipy.integrate import odeint
#import pytrajectory as pytr
#import pycartan as pc
from sympy.solvers import solve

#from pytrajectory import log
#log.console_handler.setLevel(10)

from sympy.physics.vector import init_vprinting, vlatex
init_vprinting(use_latex='mathjax', pretty_print=False)

import dill
#import ipydex

#definingymbolic variables
#q = st.symb_vector('q:2')
#qdot = st.time_deriv(q, q)
#qdd = st.time_deriv(qdot, qdot)
q0,q1=me.dynamicsymbols('q0 q1')
qdot0, qdot1=me.dynamicsymbols('q0 q1',1)
qdd0, qdd1=me.dynamicsymbols('q0 q1',2)
q=[q0, q1]
qdot=[qdot0, qdot1]
qdd=[qdd0, qdd1]
#st.make_global(q0, qdot0)

f = sm.symbols('f')
m = sm.symbols('m:2')
J = sm.symbols('J:2')
l = sm.symbols('l')
a = sm.symbols('a')
d = sm.symbols('d')
g, t = sm.symbols('g t')

# defining reference frames and points
In_frame = me.ReferenceFrame('In_frame')  #reference frame with velocity 0
O = me.Point('O')
O.set_vel(In_frame, 0)

# Mass center of the cart
C0 = me.Point('C0')
C0.set_pos(O, q[0] * In_frame.x)
C0.set_vel(In_frame, qdot[0] * In_frame.x)

#cart frame
cart_frame = In_frame.orientnew('L0', 'Axis', [q[0], In_frame.x])
cart_frame.set_ang_vel(In_frame, 0)

#endulum frame
pen_frame = In_frame.orientnew('L1', 'Axis', [ sm.pi/2-q[1], In_frame.z])
pen_frame.set_ang_vel(In_frame, qdot[1] * In_frame.z)
#mass center of pendulum
s = C0.locatenew('a', a * pen_frame.x)
s.v2pt_theory(C0, In_frame, pen_frame)

# finding velocities in inertial reference frame --> use them to find kinetic energy
v0 = C0.vel(In_frame)
v1 = s.vel(In_frame)
T = 1 / 2 * m[0] * v0.dot(v0) + 1 / 2 * m[1] * v1.dot(v1) + 1 / 2 * J[1] * qdot[1]**2

v00=qdot[0]
v11_sq=(qdot[0]+a*qdot[1]*sm.cos(q[1]))**2 + (a*qdot[1]*sm.sin(q[1]))**2

T1=1/2 * m[0] * v00**2 +1/2 * m[1] * v11_sq +1/2 *J[1]*qdot[1]**2
# potential energy
h = s.pos_from(O).dot(In_frame.y)
#V = m[0] * g * h - 1/2 * d * qdot[1]**2
V = m[0] * g * h
tau = -d * qdot[1]

params = [m[0], m[1], J[0], J[1], a, d, g]
st.make_global(params)

qq=sm.Matrix([q, qdot])
equilib_point = sm.Matrix([1.0,sm.pi/6 , 1, 1])

parameter_values = [(g, 9.81), (a, 0.2), (d, 0.0), (m[0], 3.34),
                    (m[1], 0.8512), (J[0], 0), (J[1], 0.01980)]

parameters = list(map(lambda a,b :(a,b),qq, equilib_point)) + parameter_values
param_dict=dict(parameters)

L=T-V
dL_dqdot0=sm.diff(L, qdot[0])
dL_dqdot1=sm.diff(L, qdot[1])
dL_dq0=sm.diff(L, q[0])
dL_dq1=sm.diff(L, q[1])

dt_dL_dqdot0=sm.diff(dL_dqdot0, t).simplify()
dt_dL_dqdot1=sm.diff(dL_dqdot1, t).simplify()

exp1=dt_dL_dqdot0-dL_dq0-f
exp2=dt_dL_dqdot1-dL_dq1

sol=sm.solve([exp1, exp2], [qdd0, qdd1])


In [49]:
q0dd_exp=sol[qdd0].expand()
q1dd_exp=sol[qdd1].expand()

In [50]:
#help(sm.collect)

In [51]:
collected_q0dd=sm.collect(q0dd_exp, f,evaluate=False )
collected_q1dd=sm.collect(q1dd_exp, f, evaluate=False)

In [60]:
gx=sm.Matrix([[0.0],
              [0.0],
              [collected_q0dd[f]],
              [collected_q1dd[f]]])

fx=sm.Matrix([[qdot0],
              [qdot1],
              [collected_q0dd[1].factor()],
              [collected_q1dd[1].factor()]])

xx=sm.Matrix([q0, q1, qdot0, qdot1])

In [61]:
gx.subs(param_dict)

Matrix([
[                      0.0],
[                      0.0],
[        0.264023561869032],
[0.417354137317858*sqrt(3)]])

In [62]:
fx.subs(param_dict)[2].simplify() # 0.0275104

-0.022473685586292 + 2.73495505017491*sin(0.333333333333333*pi)

In [63]:
gx_lin=gx.jacobian(xx)
fx_lin=fx.jacobian(xx)

In [69]:
sm.simplify(gx_lin*xx)

Matrix([
[                                                                                                                                         0],
[                                                                                                                                         0],
[                       -4*a**2*m1**2*(J1 + a**2*m1)*q1*sin(2*q1)/(2*J1*m0 + 2*J1*m1 + 2*a**2*m0*m1 - a**2*m1**2*cos(2*q1) + a**2*m1**2)**2],
[-a*m1*(J1*m0 + J1*m1 + a**2*m0*m1 + a**2*m1**2*cos(q1)**2 + a**2*m1**2)*q1*sin(q1)/(J1*m0 + J1*m1 + a**2*m0*m1 + a**2*m1**2*sin(q1)**2)**2]])

In [68]:
sm.simplify(fx_lin*xx)

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                    q0'],
[                                                                                                                                                                                                                                                                                                                                                                                                    q1'],
[a*m1*(-2.0*(J1 + a**2*m1)*(J1*m0 + J1*m1 + a**2*m0*m1 + a**2*m1**2*sin(q1)**2)*sin(q1)*q1'**2 + (2.0*a**2*m1**2*(1.0*J1*sin(q1)*q1'**2 + 1.0*a**2*m1*sin(q1)*q1'**2 - 0.5*a*g*m0*sin(2.0*q1))*sin(q1)*co

In [39]:
fx_lin.subs(param_dict)*xx

Matrix([
[                q0'],
[                q1'],
[5.67138717420427*q1],
[139.625927658159*q1]])

In [40]:
gx_lin.subs(param_dict)*xx

Matrix([
[0],
[0],
[0],
[0]])